# OpenCV 运动检测
本章教程使用 OpenCV 来检测画面中的变化，你可以为变化多少设置一个阈值，更改阈值，可以更改运动检测的敏感度。

本章节需要前置章节的基础。

## 准备工作
由于产品开机默认会自动运行主程序，主程序会占用摄像头资源，这种情况下是不能使用本教程的，需要结束主程序或禁止主程序自动运行后再重新启动机器人。

如果你已经禁用了机器人主程序的开机自动运行，则不需要执行下面的`结束主程序`章节。

### 结束主程序
1. 点击上方本页面选项卡旁边的 “+”号，会打开一个新的名为 Launcher 的选项卡。
2. 点击 Other 内的 Terminal，打开终端窗口。
3. 在终端窗口内输入 `bash` 后按回车。
4. 现在你可以使用 Bash Shell 来控制机器人了。
5. 输入命令： `sudo killall -9 python`。

## 例程
以下代码块可以直接运行：

1. 选中下面的代码块
2. 按 Shift + Enter 运行代码块
3. 观看实时视频窗口
4. 按 `STOP` 关闭实时视频，释放摄像头资源

### 如果运行时不能看到摄像头实时画面
- 需要点击上方的 Kernel - Shut down all kernels
- 关闭本章节选项卡，再次打开
- 点击 `STOP` 释放摄像头资源后重新运行代码块
- 重启设备

### 本章节的特性
你需要更改一些参数来调增 OpenCV 对画面中变化检测的阈值（灵敏度）`threshold`，这个阈值越低，OpenCV 对画面的变化越敏感。

### 运行
运行代码块是，你可以看到摄像头的实时画面，可以在画面前挥手，本例程会自动将出现变化的部分使用绿色的方框圈起来。

In [ ]:
import cv2
import numpy as np
from IPython.display import display, Image
import ipywidgets as widgets
import threading

import imutils # 用于简化图像处理任务的库

threshold = 2000 # 设置动态检测阈值

#  创建一个“停止”按钮来控制流程
# ================
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='square' # 按钮图标（FontAwesome 名称，不带 `fa-` 前缀）
)


# 显示函数定义，用于捕获和处理视频帧，同时进行运动检测
# ================
def view(button):
    camera = cv2.VideoCapture(-1) # 创建摄像头实例
    #设置分辨率
    camera.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    display_handle=display(None, display_id=True)
    i = 0
    
    avg = None # 用于存储平均帧
    
    while True:
        _, frame = camera.read() # 从摄像头捕获一帧图像

        img = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) # 将帧颜色从 RGB 转换为 BGR
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # 将帧转换为灰度图
        gray = cv2.GaussianBlur(gray, (21, 21), 0) # 对灰度图应用高斯模糊
        if avg is None: # 如果平均帧不存在，则创建它
            avg = gray.copy().astype("float")
            continue

        try:
            cv2.accumulateWeighted(gray, avg, 0.5) # 更新平均帧
        except:
            continue

        frameDelta = cv2.absdiff(gray, cv2.convertScaleAbs(avg)) # 计算当前帧和平均帧的差值

        # 应用阈值，找到差值图像中的轮廓
        thresh = cv2.threshold(frameDelta, 5, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=2)
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        # 遍历轮廓
        for c in cnts:
            # 如果轮廓太小，则忽略
            if cv2.contourArea(c) < threshold:
                continue
            # 计算轮廓的边界框，将其绘画到矩形框
            (mov_x, mov_y, mov_w, mov_h) = cv2.boundingRect(c)
            cv2.rectangle(frame, (mov_x, mov_y), (mov_x + mov_w, mov_y + mov_h), (128, 255, 0), 1) # 在移动区域画矩形框
            
        
        
        _, frame = cv2.imencode('.jpeg', frame) # 将处理后的帧编码为 JPEG 格式
        display_handle.update(Image(data=frame.tobytes())) # 更新显示的图像
        if stopButton.value==True: # 检查是否按下了“停止”按钮
            # picam2.close() # 如果是，则关闭摄像头
            cv2.release() # 如果是，则关闭摄像头
            display_handle.update(None) # 清除显示的图像

            
# 显示停止按钮并启动视频流显示线程
# ================
display(stopButton)
thread = threading.Thread(target=view, args=(stopButton,))
thread.start()